## Inflation analysis with token distribution

In [17]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact
import math

#### Constants

In [18]:
initial_supply = 1e9
max_supply = 1e10
total_supply = 1e10
decay_constant = 0.5

# Token Distribution Percentages
distribution_percentages = {
    "Seed Sales": 9,
    "Private Sales": 15.1,
    "Public Sales": 11.8,
    "Team": 15,
    "Foundation": 20,
    "Community & Ecosystem": 28.4,
    "Exchange Security Deposit": 0.65
}

In [19]:
# Dynamic Inflation Function
def dynamic_inflation(t, current_supply, max_supply, decay_constant):
    remaining_supply_ratio = (max_supply - current_supply) / max_supply
    inflation_rate = decay_constant * remaining_supply_ratio
    return inflation_rate

# Calculate Supply Based on Dynamic Inflation
def calculate_supply_dynamic(initial_supply, max_supply, decay_constant, time_horizon):
    supply = [initial_supply]
    for t in range(1, int(time_horizon) + 1):
        current_supply = supply[-1]
        inflation_rate = dynamic_inflation(t, current_supply, max_supply, decay_constant)
        new_supply = current_supply * (1 + inflation_rate)
        supply.append(min(new_supply, max_supply))
    return np.array(supply)

# Calculate Vesting Schedule
def calculate_vesting_schedule(tge_percentage, vesting_months, cliff_months):
    pre_seed_allocation = total_supply * (distribution_percentages["Seed Sales"] / 100)
    tge_unlocked_tokens = pre_seed_allocation * (tge_percentage / 100)
    monthly_vesting_tokens = (pre_seed_allocation - tge_unlocked_tokens) / vesting_months

    months = np.arange(0, vesting_months + cliff_months + 1)
    tokens = np.zeros_like(months, dtype=float)
    tokens[0] = tge_unlocked_tokens
    tokens[cliff_months + 1:] = monthly_vesting_tokens

    df = pd.DataFrame({
        'Month': months,
        'Tokens Unlocked': tokens,
        'Cumulative Tokens': np.cumsum(tokens)
    })
    
    return df

#### Plots

In [20]:
# Plot Vesting Schedule
def plot_vesting_schedule(tge_percentage, vesting_months, cliff_months):
    df = calculate_vesting_schedule(tge_percentage, vesting_months, cliff_months)
    
    plt.figure(figsize=(12, 6))
    plt.plot(df['Month'], df['Cumulative Tokens'], marker='o', linestyle='-', color='b')
    plt.title('DeGym Pre-Seed Token Vesting Schedule')
    plt.xlabel('Month')
    plt.ylabel('Cumulative Tokens Unlocked')
    plt.xticks(df['Month'])
    plt.grid(True)
    plt.show()
    
    display(df)

# Function to Plot Combined Data
def plot_combined_data(tge_percentage, vesting_months, cliff_months, decay_constant):
    time_horizon = 180  # 15 years in months
    
    # Calculate Vesting Schedule
    vesting_df = calculate_vesting_schedule(tge_percentage, vesting_months, cliff_months)
    
    # Calculate Inflation
    months = np.arange(0, time_horizon + 1)
    supply_dynamic = calculate_supply_dynamic(initial_supply, max_supply, decay_constant, time_horizon)
    
    # Combine Data
    initial_distribution = {k: v / 100 * initial_supply for k, v in distribution_percentages.items()}
    community_tokens = [initial_distribution["Community & Ecosystem"]]
    for t in range(1, len(supply_dynamic)):
        community_tokens.append(community_tokens[-1] + (supply_dynamic[t] - supply_dynamic[t - 1]))
    
    combined_df = pd.DataFrame({
        'Month': months,
        'Total Supply': supply_dynamic,
        'Community & Ecosystem Tokens': community_tokens
    })
    
    for category, percentage in distribution_percentages.items():
        if category != "Community & Ecosystem":
            combined_df[category + ' Tokens'] = percentage / 100 * supply_dynamic[0]
    
    # Plot Combined Data
    plt.figure(figsize=(12, 8))
    
    plt.plot(combined_df['Month'], combined_df['Total Supply'], label='Total Supply', color='tab:blue')
    plt.plot(combined_df['Month'], combined_df['Community & Ecosystem Tokens'], label='Community & Ecosystem Tokens', color='tab:green')
    
    for category, percentage in distribution_percentages.items():
        if category != "Community & Ecosystem":
            plt.plot(combined_df['Month'], combined_df[category + ' Tokens'], label=category + ' Tokens')
    
    plt.xlabel('Time (Months)')
    plt.ylabel('Tokens')
    plt.title('Token Distribution Over Time with Inflation')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    display(combined_df)

In [21]:
interact(plot_combined_data,
         tge_percentage=widgets.IntSlider(min=0, max=100, step=5, value=10, description='TGE %'),
         vesting_months=widgets.IntSlider(min=1, max=24, step=1, value=12, description='Vesting Months'),
         cliff_months=widgets.IntSlider(min=0, max=12, step=1, value=3, description='Cliff Months'),
         decay_constant=widgets.FloatSlider(min=0.01, max=1.0, step=0.01, value=decay_constant, description='Decay Constant')
        );


interactive(children=(IntSlider(value=10, description='TGE %', step=5), IntSlider(value=12, description='Vesti…